### Descarga Realtime Alpha Vantage
###### Descarga cada 10 minutos

In [ ]:
import pandas as pd
import numpy as np
import time
#import matplotlib.pyplot as plt
import os
from datetime import date, datetime
#from dateutil.relativedelta import relativedelta

In [ ]:
#leer archivo
# Verificar si el archivo existe
#ruta_archivo = r"D:\traderxpro\data\dataxh.txt"
ruta_archivo='notebooks/data/dataxh.txt'
if os.path.exists(ruta_archivo):
    # Cargar el archivo
    df = pd.read_csv(ruta_archivo, sep='\t')
    print("Archivo cargado correctamente.")
else:
    # Crear un DataFrame vacío
    df = pd.DataFrame()
    print("Archivo no existe. Se creó un DataFrame vacío.")

Archivo cargado correctamente.


In [39]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [41]:
max_day = (df['datetime'].max()).date()

In [43]:
max_day

datetime.date(2025, 5, 7)

In [45]:
#today = date.today()
max_day_str = str(max_day)
tickers = [
'SPY',
'AAPL',
'META'
]

In [47]:
#Funcion limpiar OUTLIERS
def df_limpiar(df_eval):
    #dataframe para obtener medias sin incluir fecha actual
    df_eval2 = df_eval[df_eval['datetime']<max_day_str].tail(100)
    
    meandif1 = df_eval2['dif1'].mean()
    stddif1 = df_eval2['dif1'].std()
    topdif1 = meandif1 + stddif1 * 1.96

    meandif2 = df_eval2['dif2'].mean()
    stddif2 = df_eval2['dif2'].std()
    topdif2 = meandif2 + stddif2 * 1.96
    
    #botdif = meandif - stddif * 3
    copydf = df_eval.copy()
    
    #df_h.apply(lambda row: row['High'] - row['Close'] if row['Close'] > row['Open'] else row['High'] - row['Open'], axis=1)
    copydf['ind1'] = copydf.apply(lambda row: 1 if row['dif1'] > topdif1 else 0, axis=1)
    copydf['ind2'] = copydf.apply(lambda row: 1 if row['dif2'] > topdif2 else 0, axis=1)

    #hoy = datetime.now()    
    #mes_actual = hoy.replace(day=1)
    #mes_fin = mes_actual - relativedelta(months=3)
    #hace_12_meses = mes_actual - relativedelta(months=12)
    #cuartil 3
    q3_dif1 = copydf[copydf['ind1']!=1]['dif1'].quantile(0.75)
    q3_dif2 = copydf[copydf['ind2']!=1]['dif2'].quantile(0.75)
    
    #meanfin1 = copydf[(copydf['ind1']==1) & (copydf['datetime'] >= hace_12_meses) & (copydf['datetime'] < mes_fin)]['dif1'].mean()
    #meanfin2 = copydf[(copydf['ind2']==1) & (copydf['datetime'] >= hace_12_meses) & (copydf['datetime'] < mes_fin)]['dif2'].mean()

    print("q3_dif1:",q3_dif1)
    print("q3_dif2:",q3_dif2)

    copydf['High2']=copydf.apply(
        lambda row: row['Close'] + q3_dif1 if (row['Close'] > row['Open']) & (row['ind1']==1) & (row['eval']==0)  else 
        row['Open'] + q3_dif1 if (row['Close'] <= row['Open']) & (row['ind1']==1) & (row['eval']==0) else
        row['High'], axis=1)
    copydf['Low2']=copydf.apply(
        lambda row: row['Open'] - q3_dif2 if (row['Close'] > row['Open']) & (row['ind2']==1) & (row['eval']==0) else 
        row['Close'] - q3_dif2 if (row['Close'] <= row['Open']) & (row['ind2']==1) & (row['eval']==0) else
        row['Low'], axis=1)
   
    #copydf = copydf.drop(copydf[copydf['dif1'] > topdif1].index)
    #copydf = copydf.drop(copydf[copydf['dif2'] > topdif2].index)
    return copydf


In [49]:
df_new = pd.DataFrame()
for ticker in tickers:
    #Informacion anterior por ticker
    df_ticker = df[(df['datetime'] <  max_day_str) & (df['companyName']==ticker)]
    df_ticker = df_ticker.reset_index(drop=True)
    df_ticker['datetime'] = pd.to_datetime(df_ticker['datetime'])
    #crear columnas de diferencia
    df_ticker['dif1'] = df_ticker.apply(lambda row: row['High'] - row['Close'] if row['Close'] > row['Open'] else row['High'] - row['Open'], axis=1)
    df_ticker['dif2'] = df_ticker.apply(lambda row: row['Open'] - row['Low'] if row['Close'] > row['Open'] else row['Close'] - row['Low'], axis=1)
    df_ticker['eval'] = 1 #no se limpiara
    #Aplha vantage descarga intraday
    url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=' + str(ticker)+ '&interval=60min&entitlement=delayed&apikey=HR9C4MJVM7O4UUCM&datatype=csv'
    company = pd.read_csv(url, sep=',')         
    company = company[['close','high','low','open','volume','timestamp']]
    company=company.rename(columns={"open": "Open","high": "High","low": "Low","close": "Close","volume": "Volume","timestamp": "datetime"})        
    company['companyName']=ticker
    company['datetime'] = pd.to_datetime(company['datetime'])
    company['datetime'] = company['datetime'].dt.tz_localize('UTC')
    #company['datetime'] = company['datetime'].dt.tz_localize('US/Eastern')
    company_lastday = company[company['datetime']>=max_day_str]
    company_lastday = company_lastday.sort_values(by=['datetime'], ascending=[True])
    
    #crear columnas diferencia de las colas (se evaluara el ultimo dia)
    company_lastday['dif1'] = company_lastday.apply(lambda row: row['High'] - row['Close'] if row['Close'] > row['Open'] else row['High'] - row['Open'], axis=1)
    company_lastday['dif2'] = company_lastday.apply(lambda row: row['Open'] - row['Low'] if row['Close'] > row['Open'] else row['Close'] - row['Low'], axis=1)
    company_lastday['eval'] = 0 # se limpiara    
    df_ticker = pd.concat([df_ticker, company_lastday],ignore_index=True)    
    company_final = df_limpiar(df_ticker)    
    df_new = pd.concat([df_new, company_final],ignore_index=True)

q3_dif1: 0.537811279296875
q3_dif2: 0.635009765625
q3_dif1: 0.4080772399902344
q3_dif2: 0.43499755859375
q3_dif1: 1.2649917602538636
q3_dif2: 1.3235321044921875


In [51]:
df_new['High'] = np.where(df_new['eval'] == 0, df_new['High2'], df_new['High'])
df_new['Low'] = np.where(df_new['eval'] == 0, df_new['Low2'], df_new['Low'])

In [53]:
df_new = df_new.drop(columns=['dif1', 'dif2', 'eval', 'ind1', 'ind2', 'High2','Low2' ])

In [57]:
df.shape[0]

10489

In [59]:
df_new.shape[0]

10489

In [28]:
#from bokeh.plotting import figure, show, column

In [61]:
#CODIGO DE REVISION CARLOS
#dfpl = df_new.query("companyName=='SPY'")
##dfpl = df.iloc[0:1000]
#dfpl.reset_index(drop=True, inplace=True)
#dfpl["Datetime_str"] = dfpl["datetime"].astype(str)
#
#p = figure(width=2500, height=500,
#        title="RCB",
#        background_fill_color="#efefef",
#        tooltips=[("datetime", "@Datetime_str"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close"), ("ticker","@companyName")]
#        )
#p.xaxis.major_label_orientation = 0.8 # radians
#p.x_range.range_padding = 0.05
##p.xaxis.axis_line_join = "bevel" # radians
#p.xaxis.axis_line_width = 2
#
## map dataframe indices to date strings and use as label overrides
#p.xaxis.major_label_overrides = {
#    #i: date.strftime('%b %d') for i, date in zip(df.index, df["datetime"])
#    i: date.strftime('%b %d %T') for i, date in zip(dfpl.index, dfpl["datetime"])
#}
#
#p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)
#
#inc = dfpl.query("Close>Open")
#dec = dfpl.query("Open>Close")
#
#p.vbar(    
#    x="index",
#    width=0.6,
#    bottom="Open",
#    top="Close",
#    fill_color="red",
#    line_color="red",    
#    source=dec   
#)
#
#
#p.vbar(    
#    x="index",
#    width=0.6,
#    bottom="Open",
#    top="Close",
#    fill_color="green",
#    line_color="green", 
#    source=inc   
#)
#
##p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
##           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])
#
##p.scatter(x="index", y="pivotLow", marker="circle", size=10,
##           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])
#
#show(p)

In [ ]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA

#path = r'D:\traderxpro\data\dataxh.txt'

path='notebooks/data/dataxh.txt'
folder=os.path.dirname(path)

if not os.path.exists(folder):
    os.makedirs(folder)



# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_new.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)